In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [5]:
data_rome = pd.read_csv('data_rome.csv')

In [6]:
data_rome = data_rome.drop(['Unnamed: 0'], axis=1)

In [7]:
from scipy import stats

# Assuming df is the name of your dataframe
z_scores = np.abs(stats.zscore(data_rome))

# Remove rows where any of the columns have a z-score greater than 3
data_rome_cleaned = data_rome[(z_scores < 3).all(axis=1)]

In [8]:
data_rome_cleaned.corr()

,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,lng,lat,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
realSum,1.000000,0.487134,-0.087640,0.096673,0.043677,0.087171,0.347868,-0.207760,0.086983,0.535250,0.513424,-0.195773,0.146476,NaN,-0.487286,0.057993,0.043406
person_capacity,0.487134,1.000000,-0.086882,-0.015288,-0.019317,0.031851,0.589228,0.024929,0.035055,0.117632,0.122476,-0.091520,-0.002471,NaN,-0.586244,0.044861,0.006741
multi,-0.087640,-0.086882,1.000000,-0.566030,0.025492,0.039730,-0.048951,0.041935,0.001593,-0.087310,-0.086055,0.029668,-0.005942,NaN,0.149969,0.084257,-0.001783
biz,0.096673,-0.015288,-0.566030,1.000000,-0.127480,-0.188529,-0.058340,-0.206393,-0.040123,0.246290,0.241110,-0.033129,0.076242,NaN,0.094869,-0.162109,0.010539
cleanliness_rating,0.043677,-0.019317,0.025492,-0.127480,1.000000,0.593774,0.013465,0.085704,-0.010344,-0.087819,-0.104327,-0.008747,-0.024872,NaN,0.004139,0.339214,-0.001911
guest_satisfaction_overall,0.087171,0.031851,0.039730,-0.188529,0.593774,1.000000,0.042323,0.102963,0.023944,-0.067593,-0.079381,-0.034301,-0.032702,NaN,-0.064100,0.385490,-0.010151
bedrooms,0.347868,0.589228,-0.048951,-0.058340,0.013465,0.042323,1.000000,0.040498,-0.008450,-0.012907,-0.012075,-0.008999,-0.002747,NaN,-0.288496,0.043586,0.006477
dist,-0.207760,0.024929,0.041935,-0.206393,0.085704,0.102963,0.040498,1.000000,0.281483,-0.491386,-0.457268,-0.332433,-0.232052,NaN,-0.029300,0.032788,-0.008606
metro_dist,0.086983,0.035055,0.001593,-0.040123,-0.010344,0.023944,-0.008450,0.281483,1.000000,0.026581,0.182829,-0.357747,-0.017889,NaN,-0.118335,-0.036685,-0.028364
attr_index_norm,0.535250,0.117632,-0.087310,0.246290,-0.087819,-0.067593,-0.012907,-0.491386,0.026581,1.000000,0.883685,-0.291100,0.190046,NaN,-0.195272,-0.005314,0.008772


In [9]:
data_rome_cleaned_reg = data_rome_cleaned.drop(['host_is_superhost_True', 'lat', 'lng'], axis=1)

In [10]:
x = data_rome_cleaned_reg.drop('realSum', axis=1)
y = data_rome_cleaned_reg.realSum

In [11]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,weekend_yes
count,6108.000000,6108.000000,6108.000000,6108.000000,6108.000000,6108.000000,6108.000000,6108.000000,6108.000000,6108.000000,6108.0,6108.000000,6108.000000
mean,3.284054,0.394237,0.328585,9.590537,93.808612,1.174689,2.944363,0.772258,10.266637,24.891672,0.0,0.396038,0.500491
std,1.262512,0.488726,0.469737,0.588762,5.674935,0.452167,1.553782,0.570840,5.618847,12.482929,0.0,0.489113,0.500041
min,2.000000,0.000000,0.000000,8.000000,70.000000,0.000000,0.040000,0.010000,2.225460,6.293903,0.0,0.000000,0.000000
25%,2.000000,0.000000,0.000000,9.000000,91.000000,1.000000,1.840000,0.310000,5.577323,14.670689,0.0,0.000000,0.000000
50%,3.000000,0.000000,0.000000,10.000000,95.000000,1.000000,2.790000,0.590000,9.077761,21.476482,0.0,0.000000,1.000000
75%,4.000000,1.000000,1.000000,10.000000,98.000000,1.000000,3.950000,1.170000,13.857394,33.212708,0.0,1.000000,1.000000
max,6.000000,1.000000,1.000000,10.000000,100.000000,2.000000,7.820000,2.710000,29.860821,65.296342,0.0,1.000000,1.000000


In [12]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     582.8
Date:                Thu, 09 Mar 2023   Prob (F-statistic):               0.00
Time:                        22:07:11   Log-Likelihood:                -33306.
No. Observations:                6108   AIC:                         6.664e+04
Df Residuals:                    6095   BIC:                         6.673e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [13]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [11]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     635.9
Date:                Wed, 08 Mar 2023   Prob (F-statistic):               0.00
Time:                        12:20:49   Log-Likelihood:                -33306.
No. Observations:                6108   AIC:                         6.664e+04
Df Residuals:                    6096   BIC:                         6.672e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        196.2539      0.723    271.276      0.0

In [12]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-111.98283228427181

In [13]:
model.score(x_train, y_train)

0.5343202791912423

In [14]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [15]:
mse(y_test,y_pred)

2858.751874515323

In [16]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

53.46729724341154

In [17]:
##prediction on the train set
mse(y_train,y_pred_train)

3190.506158330639

In [18]:
mae(y_test,y_pred)

39.605308846134896

In [19]:
R2=r2_score(y_test,y_pred)
R2

0.5458417170504972

In [20]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.5431490790092947

In [21]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.5343005131944081, Test -> 0.5460613978974447


In [22]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.5195580879407876, Test -> 0.5390187577093746


In [23]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 6108 rows and 12 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.53
The coefficient of determination for the train data is R2=1.00


In [24]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.45657949 0.49508674 0.48923056 0.4576508  0.47155779]
Score stats: 0.47 accuracy with a standard deviation of 0.02


In [25]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.4729460160865406, 'Linear Regression': 0.5325542467711283, 'KNN': 0.5279506035701257}


In [26]:
data_rome_cleaned.to_excel('data_rome.xlsx')